In [21]:
import os
import pandas as pd
from kafka import KafkaProducer
from tqdm import tqdm

In [22]:
DATA_INPUT_FILES = ["../data/customer.csv"]#,"../data/date.csv","../data/product.csv","../data/reseller.csv","../data/sales.csv","../data/sales_order.csv","../data/sales_territory.csv"]
KAFKA_SERVER = "20.109.88.59:9092"
KAFKA_TOPICS = ["prueba.customer"]#,"prueba.date","prueba.product","prueba.reseller","prueba.sales","prueba.sales-order","prueba.sales-terrritory"]

In [23]:
producer = KafkaProducer(bootstrap_servers = [KAFKA_SERVER])

In [24]:
def send_2_kafka(input_data_path:str, kafka_topic_name:str) -> int:
    #me aseguro de que el archivo si existe
    assert os.path.exists(input_data_path), "El path al archivo de datos de entrada es erroneo, o el archivo no existe."
    file = os.path.splitext(input_data_path)
    #me aseguro de que la extension del archivo es CSV
    assert file[1] == ".csv", f"El archivo {file[0]} debe de ser de extension CSV."
    try:
        df = pd.read_csv(input_data_path)
        print(df.iloc[0:3])
        progress_bar = tqdm(total=len(df), desc=f"Loading to {kafka_topic_name} topic")
        for _,row in df.iterrows():
            message = row.to_json()
            producer.send(kafka_topic_name, value=message.encode())
            producer.flush() #asegurarse de que todos los datos hayan sido escritos
            progress_bar.update(1)
        producer.close()
        return 0
    except ValueError as error:
        print(error)
        return 1

In [25]:
def main():
    #assert len(DATA_INPUT_FILES) == len(KAFKA_TOPICS), "Deben de haber tantos topicos como archivos de entrada hallan."
    for data_file, kafka_topic in zip(DATA_INPUT_FILES,KAFKA_TOPICS):
        send_2_kafka(
            input_data_path=data_file,
            kafka_topic_name=kafka_topic,
        )

In [ ]:
if __name__ == "__main__":
    main()

Loading to prueba.customer topic:  18%|█▊        | 3367/18485 [04:50<20:39, 12.20it/s]